In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

In [8]:
driver = webdriver.Chrome()
driver.wait = WebDriverWait(driver, 5)
driver.get(f"https://blast.ncbi.nlm.nih.gov/"
           f"Blast.cgi?PROGRAM=blastp&PAGE_TYPE=BlastSearch&LINK_LOC=blasthome")

box = driver.wait.until(EC.presence_of_element_located(
            (By.NAME, "QUERY")))
button = driver.wait.until(EC.element_to_be_clickable(
            (By.CLASS_NAME, "blastbutton")))
organism = driver.wait.until(EC.presence_of_element_located(
            (By.NAME, "EQ_MENU")))
add_options = driver.wait.until(EC.element_to_be_clickable(
            (By.XPATH, '//*[@id="algPar"]/span[2]')))
psi_blast = driver.wait.until(EC.element_to_be_clickable(
            (By.XPATH, '//*[@id="psiBlast"]')))

box.send_keys(f'MKLLEEWKSERYYLKVGEMFVKEVSFDVISL'
              f'TANQSYADEFDKNRGTFKDVYLHTKLHAEKN'
              f'NLGKVRLIKIVRTVTEVEMDADKLTVEVLEQYVKEEKEKHNL')
organism.send_keys(f'Viruses (taxid:10239)')
psi_blast.click()
#add_options.click()
#button.click()

In [ ]:
element.get_attribute('innerHTML')